# Yahtzee

__Fully Connected Networks__

_By Marnick van der Arend & Jeroen Smienk_

![Yahtzee](yahtzee.png)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

__MODEL_PATH = 'yahtzee-models'
__TENSOR_LOG_DIR = 'yahtzee-logs'

## Dataset

Let's start with looking at the provided dataset:

It consists of 5832 'dice throws' and the label that describes that type of throw.

The existing Yahtzee labels are: `nothing`, `small-straight`, `three-of-a-kind`, `large-straight`,
 `full-house`, `four-of-a-kind`, and `yathzee`.
 
 Name | Description
--- | ---
3-of-a-kind | Three dice the same
4-of-a-kind | Four dice the same
Full-house | Three of one number and two of another
Small-straight | Four sequential dice
Large-straight | Five sequential dice
Yahtzee | All five dice the same
Nothing | None of the above combinations

In [2]:
df = pd.read_csv('yahtzee-dataset.csv')
df.head(5)

,dice1,dice2,dice3,dice4,dice5,label
0,3,6,6,2,5,nothing
1,3,6,1,3,4,nothing
2,2,2,5,5,3,nothing
3,1,3,6,6,1,nothing
4,1,4,6,3,5,small-straight


Let's see how the labels are distributed in the dataset:

In [3]:
df.label.value_counts() / len(df.index)

nothing            0.663237
three-of-a-kind    0.153635
small-straight     0.094136
full-house         0.037894
large-straight     0.030521
four-of-a-kind     0.019890
yathzee            0.000686
Name: label, dtype: float64

We see that 66.3% of all data is `nothing` throws. This makes sense of course because how harder the dice throws get, the less chance exists they occur.

The most difficult throw (`yahtzee`) only occurs 0.069%! This might make it hard for any of our models to really learn what a `yahtzee` throw is.

### One-Hot Encoding

In order to classify these categorical labels, we have to 'one-hot encode' them:

In [4]:
one_hot_df = pd.get_dummies(df, prefix=['label'])
one_hot_df.head(10)

,dice1,dice2,dice3,dice4,dice5,label_four-of-a-kind,label_full-house,label_large-straight,label_nothing,label_small-straight,label_three-of-a-kind,label_yathzee
0,3,6,6,2,5,0,0,0,1,0,0,0
1,3,6,1,3,4,0,0,0,1,0,0,0
2,2,2,5,5,3,0,0,0,1,0,0,0
3,1,3,6,6,1,0,0,0,1,0,0,0
4,1,4,6,3,5,0,0,0,0,1,0,0
5,4,1,4,3,1,0,0,0,1,0,0,0
6,4,4,4,6,2,0,0,0,0,0,1,0
7,3,2,5,6,3,0,0,0,1,0,0,0
8,3,4,3,6,2,0,0,0,1,0,0,0
9,3,3,1,5,4,0,0,0,1,0,0,0


Before we can train any model, we have to split the data and the labels into X and Y after shuffling them:

In [60]:
def get_shuffled_xy(one_hot_df):
    shuffled = one_hot_df.sample(frac=1.)
    return shuffled.iloc[:,:5].copy(), shuffled.iloc[:,5:].copy()

X, Y = get_shuffled_xy(one_hot_df)
X.head(5)

,dice1,dice2,dice3,dice4,dice5
379,4,4,3,3,1
2417,4,3,5,6,2
228,5,3,3,1,5
45,1,1,2,4,3
4560,1,3,1,2,6


We also split the dataset into a 85:15 split for training and validating the model:

In [64]:
def get_validation_split(x, y, frac):
    split = int(len(x.index) * frac)
    return x.iloc[:split], y.iloc[:split], x.iloc[split:], y.iloc[split:]

X_train, Y_train, X_valid, Y_valid = get_validation_split(X, Y, .85)


In [65]:
def get_test_split(x_train, y_train, frac):
    split = int(len(x_train.index) * frac)
    return x_train.iloc[:split], y_train.iloc[:split], x_train.iloc[split:], y_train.iloc[split:]
    
X_train, Y_train, X_test, Y_test = get_validation_split(X_train, Y_train, .85)


In [66]:
print('Split X (train, test, validation):', X_train.shape, X_test.shape, X_valid.shape)
print('Split Y (train, test, validation):', Y_train.shape, Y_test.shape, Y_valid.shape)

Split X (train, test, validation): (4213, 5) (744, 5) (875, 5)
Split Y (train, test, validation): (4213, 7) (744, 7) (875, 7)


We define a function that returns a random batch of a certain size. This batch is used in training to train faster.

In [7]:
def get_batch(data, labels, batch_size):
    x_batch = data.sample(frac=batch_size / len(data.index))
    return x_batch, labels.loc[x_batch.index]

## Models

We designed several models:

_Note: all models have a last layers without an activation function, because the SoftMax is done in the cross entropy calculation._

rank | name | layers | score
--- | --- | --- | ---
5 | model_8 | (64, Tanh, Drop=.2) (128, Tanh, Drop=.3) (256, Tanh, Drop=.4) (512, Tanh, Drop=.5) (64, Tanh) | 0.93714285
4 | model_7 | (64, ReLU) (128, ReLU) (256, ReLU) (512, ReLU, Drop=.3) (64, ReLU) | 0.94057140
3 | model_6 | (200, Tanh) (300, Tanh) (600, Tanh) | ±0.97
1 | model_5_2 | (600, Tanh, Drop=.3) (300, Tanh, Drop=.3) (200, Tanh, Drop=.3) | ±0.976
2 | model_5 | (600, Tanh) (300, Tanh) (200, Tanh) | 0.97942860
6 | model_4 | (128, Tanh) (64, Tanh) (32, Tanh) | 0.87771430
7 | model_3 | (12, ReLU) (24, ReLU) (48, ReLU, Drop=.1) (96, ReLU) | 0.73028570
8 | model_2 | (128, ReLU) (64, ReLU) (32, ReLU) | 0.82400000
9 | model_1 | (128, Sigmoid) | ±0.65

The best model uses dropout to prevent overfitting and the ReLU activation function which proved to perform better.

### Metric Graphs

__Legend:__

- Model 1: BLUE
- Model 2: RED
- Model 3: LIGHT BLUE
- Model 4: PINK
- Model 5: GREEN
- Model 6: GRAY
- Model 7: ORANGE
- Model 8: ORANGE

#### Batch Accuracy Graph

![Batch Accuracy](yahtzee-acc.png)

#### Batch Loss Graph

![Batch Loss](yahtzee-loss.png)

In [23]:
def model_1(x, output_shape):
    """
    Single hidden layer with 128 neurons and Sigmoid activation function.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.sigmoid)
    return tf.layers.dense(l_1, units=output_shape, activation=None)

In [24]:
def model_2(x, output_shape):
    """
    Three hidden layers with different amounts of neurons and relu activation functions.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=64, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=32, activation=tf.nn.relu)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [25]:
def model_3(x, output_shape):
    """
    Six hidden layers with different amounts of neurons and 
    relu activation functions and 2 dropout layers.
    """
    l_1 = tf.layers.dense(x, units=12, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=24, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=48, activation=tf.nn.relu)
    d_3 = tf.layers.dropout(l_3, rate=.1)
    l_4 = tf.layers.dense(d_3, units=96, activation=tf.nn.relu)
    return tf.layers.dense(l_4, units=output_shape, activation=None)

In [26]:
def model_4(x, output_shape):
    """
    Three hidden layers with different amounts of neurons and relu activation functions.
    """
    l_1 = tf.layers.dense(x, units=128, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=64, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=32, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [27]:
def model_5(x, output_shape):
    """
    High number of neurons in layers, decreasing per layer; no dropout; TanH activation.
    """
    l_1 = tf.layers.dense(x, units=600, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=300, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=200, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [53]:
def model_5_2(x, output_shape):
    """
    High number of neurons in layers, decreasing per layer; consistent dropout; TanH activation.
    """
    l_1 = tf.layers.dense(x, units=600, activation=tf.nn.tanh)
    d_1 = tf.layers.dropout(l_1, rate=.3)
    l_2 = tf.layers.dense(d_1, units=300, activation=tf.nn.tanh)
    d_2 = tf.layers.dropout(l_2, rate=.3)
    l_3 = tf.layers.dense(d_2, units=200, activation=tf.nn.tanh)
    d_3 = tf.layers.dropout(l_3, rate=.3)
    return tf.layers.dense(d_3, units=output_shape, activation=None)

In [54]:
def model_6(x, output_shape):
    """
    High number of neurons in layers, increasing per layer; no dropout; TanH activation.
    """
    l_1 = tf.layers.dense(x, units=200, activation=tf.nn.tanh)
    l_2 = tf.layers.dense(l_1, units=300, activation=tf.nn.tanh)
    l_3 = tf.layers.dense(l_2, units=600, activation=tf.nn.tanh)
    return tf.layers.dense(l_3, units=output_shape, activation=None)

In [55]:
def model_7(x, output_shape):
    """
    High number of neurons in layers; ReLU activation; single dropout layer.
    """
    l_1 = tf.layers.dense(x, units=64, activation=tf.nn.relu)
    l_2 = tf.layers.dense(l_1, units=128, activation=tf.nn.relu)
    l_3 = tf.layers.dense(l_2, units=256, activation=tf.nn.relu)
    l_4 = tf.layers.dense(l_3, units=512, activation=tf.nn.relu)
    d_4 = tf.layers.dropout(l_4, rate=.3)
    l_5 = tf.layers.dense(d_4, units=64, activation=tf.nn.relu)
    return tf.layers.dense(l_5, units=output_shape, activation=None)

In [56]:
def model_8(x, output_shape):
    """
    More layers; TanH activation; increasing amount of dropout.
    """
    l_1 = tf.layers.dense(x, units=64, activation=tf.nn.tanh)
    d_1 = tf.layers.dropout(l_1, rate=.2)
    l_2 = tf.layers.dense(d_1, units=128, activation=tf.nn.tanh)
    d_2 = tf.layers.dropout(l_2, rate=.3)
    l_3 = tf.layers.dense(d_2, units=256, activation=tf.nn.tanh)
    d_3 = tf.layers.dropout(l_3, rate=.4)
    l_4 = tf.layers.dense(d_3, units=512, activation=tf.nn.tanh)
    d_4 = tf.layers.dropout(l_4, rate=.5)
    l_5 = tf.layers.dense(d_4, units=64, activation=tf.nn.tanh)
    return tf.layers.dense(l_5, units=output_shape, activation=None)

We start with the placeholder for our 5-dice input and 7-class output and choose a model:

In [57]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, X.shape[1]], name='x')
y = tf.placeholder(tf.float32, shape=[None, Y.shape[1]], name='y')

model_fn = model_5_2
y_pred = model_fn(x, Y.shape[1])

## Train, Test, Validate

We choose an optimizer and define a loss functon and the accuracy metric:

In [58]:
# Loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_pred)
loss_fn = tf.reduce_mean(cross_entropy)

# Optimizer minimizes the loss
optimizer = tf.train.AdamOptimizer(learning_rate=.001).minimize(loss_fn)

# Accuracy metric
#   checks if the indices of the highest values in the real 
#   and predicted arrays are equal
prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

We train the model using a certain batch size and for a number of iterations while posting scalars to TensorBoard:

In [59]:
iters = 5000
train_batch_size = 100

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

session = tf.Session()
with session:
    session.run(init_op)

    # Defining the metrics we want to log in TensorBoard
    sum_loss_train = tf.summary.scalar('loss_train', loss_fn)
    sum_loss_test = tf.summary.scalar('loss_test', loss_fn)
    sum_acc_train = tf.summary.scalar('acc_train', accuracy)
    sum_acc_test = tf.summary.scalar('acc_test', accuracy)
    tf.summary.merge_all()
    writer = tf.summary.FileWriter(os.path.join(__TENSOR_LOG_DIR, model_fn.__name__), session.graph)

    # Start training for a certain number of iterations
    for i in range(iters):
        # Every iteration we get a random batch of the training data
        x_batch, y_batch = get_batch(X_train, Y_train, train_batch_size)
        # We train the model by providing the 'optimizer' variable to the run function.
        # We also want to calculate the accuracy and loss TensorBoard metrics
        loss_val, _, acc_val, sum_1, sum_2 = session.run([loss_fn, optimizer, accuracy, 
                                                          sum_loss_train, sum_acc_train], 
                                                         feed_dict={x: x_batch, y: y_batch})
        # Write the metrics to TensorBoard
        writer.add_summary(sum_1, global_step=i)
        writer.add_summary(sum_2, global_step=i)

        # Validate every 50 iterations
        if i % 50 == 0:
            # DO NOT PROVIDE THE 'optimzer' VARIABLE HERE
            # ELSE THE MODEL WILL TRAIN ON THE TEST DATA
            acc_val, sum_1, sum_2 = session.run([accuracy, sum_loss_test, sum_acc_test], 
                                                feed_dict={x: X_test, y: Y_test})
            # Write the metrics to TensorBoard
            writer.add_summary(sum_1, global_step=i)
            writer.add_summary(sum_2, global_step=i)
            print('Testing - i:', i+1, ' Accuracy:', acc_val)
    

    # Validate the model with unseen data
    acc_val = session.run([accuracy], feed_dict={x: X_valid, y: Y_valid})
    print('Validation accuracy:', acc_val)
    
    # Save the model
    saver.save(session, '{}.ckpt'.format(os.path.join(__MODEL_PATH, model_fn.__name__, model_fn.__name__)))
    

Testing - i: 1  Accuracy: 0.63844085
Testing - i: 51  Accuracy: 0.63844085
Testing - i: 101  Accuracy: 0.63844085
Testing - i: 151  Accuracy: 0.641129
Testing - i: 201  Accuracy: 0.63844085
Testing - i: 251  Accuracy: 0.64784944
Testing - i: 301  Accuracy: 0.6357527
Testing - i: 351  Accuracy: 0.6491935
Testing - i: 401  Accuracy: 0.63978493
Testing - i: 451  Accuracy: 0.6518817
Testing - i: 501  Accuracy: 0.63709676
Testing - i: 551  Accuracy: 0.6532258
Testing - i: 601  Accuracy: 0.65994626
Testing - i: 651  Accuracy: 0.66263443
Testing - i: 701  Accuracy: 0.6814516
Testing - i: 751  Accuracy: 0.68682796
Testing - i: 801  Accuracy: 0.6989247
Testing - i: 851  Accuracy: 0.7177419
Testing - i: 901  Accuracy: 0.7405914
Testing - i: 951  Accuracy: 0.78494626
Testing - i: 1001  Accuracy: 0.79435486
Testing - i: 1051  Accuracy: 0.8346774
Testing - i: 1101  Accuracy: 0.8427419
Testing - i: 1151  Accuracy: 0.83870965
Testing - i: 1201  Accuracy: 0.8467742
Testing - i: 1251  Accuracy: 0.87096

## Loading a Saved Model

We load the model that worked best:

In [67]:
model_to_load = model_5_2

load_path = '{}.ckpt'.format(os.path.join(__MODEL_PATH, 
                                          model_to_load.__name__, 
                                          model_to_load.__name__))

In [68]:
with tf.Session() as saved_session:
    tf.train.Saver().restore(saved_session, load_path)

    # Validate the model with unseen data
    acc_val = saved_session.run([accuracy], feed_dict={x: X_valid, y: Y_valid})

    # Print test metrics
    print('Validation accuracy imported {}: {}'.format(model_to_load.__name__, acc_val))

INFO:tensorflow:Restoring parameters from yahtzee-models/model_5_2/model_5_2.ckpt
Validation accuracy imported model_5_2: [0.9977143]


## Conclusion

### Dataset

- The dataset contains 5832 rows which is enough for this deep learning excersice.
- The distribution of the labels may cause the model to not optimally learn the very rare throws like `yahtzee`.

### Model

#### Sizes

#### Activation Functions

We looked at Sigmoid, ReLU and TanH. Sigmoid performed less well than ReLU and TanH. To our surprise TanH worked best.